# Advanced Features and CLI Tools
## EPyR Tools Tutorial 05: Command-Line Interface and Automation

Welcome to the final tutorial in the EPyR Tools series! This notebook covers advanced features including the command-line interface (CLI), batch processing, automation workflows, and integration capabilities.

### What You'll Learn

- **CLI Commands**: Complete command-line toolkit
- **Batch Processing**: Automated analysis of multiple files
- **FAIR Data Export**: Standards-compliant data conversion
- **Configuration Management**: Customizing EPyR Tools behavior
- **Performance Optimization**: Memory management and caching
- **Plugin Architecture**: Extending functionality
- **Integration Workflows**: Connecting with other tools

### Prerequisites

- Complete Tutorials 01-04
- Command-line familiarity
- Understanding of batch processing concepts

## 1. EPyR Tools CLI Overview

EPyR Tools provides a comprehensive command-line interface with 8 main commands. Let's explore the available tools:

In [ ]:
# First, let's check our EPyR Tools installation and CLI availability
import subprocess
import os

# Check if CLI commands are available
cli_commands = [
    'epyr-info',
    'epyr-load', 
    'epyr-convert',
    'epyr-baseline',
    'epyr-plot',
    'epyr-batch',
    'epyr-config',
    'epyr-validate'
]

print("EPyR Tools CLI Commands Availability:")
print("=" * 40)

for cmd in cli_commands:
    try:
        result = subprocess.run([cmd, '--help'], 
                              capture_output=True, text=True, timeout=10)
        if result.returncode == 0:
            print(f"✅ {cmd:<15} - Available")
        else:
            print(f"❌ {cmd:<15} - Error: {result.stderr.strip()[:50]}")
    except FileNotFoundError:
        print(f"❌ {cmd:<15} - Not found in PATH")
    except subprocess.TimeoutExpired:
        print(f"⏱️ {cmd:<15} - Timeout (likely available)")
    except Exception as e:
        print(f"❓ {cmd:<15} - Unknown error: {str(e)[:30]}")

## 2. System Information and Diagnostics

The `epyr-info` command provides comprehensive system diagnostics:

In [ ]:
# Get basic system information
try:
    result = subprocess.run(['epyr-info'], capture_output=True, text=True, timeout=15)
    if result.returncode == 0:
        print("EPyR Tools System Information:")
        print("=" * 40)
        print(result.stdout)
    else:
        print(f"Error running epyr-info: {result.stderr}")
except Exception as e:
    print(f"Could not run epyr-info: {e}")
    
    # Fallback: get info programmatically
    import epyr
    import sys
    import platform
    
    print("\nFallback System Information:")
    print("=" * 30)
    print(f"EPyR Tools Version: {epyr.__version__}")
    print(f"Python Version: {sys.version.split()[0]}")
    print(f"Platform: {platform.system()} {platform.machine()}")
    print(f"Available Modules:")
    print(f"  - Data Loading: {hasattr(epyr, 'eprload')}")
    print(f"  - Baseline Correction: {hasattr(epyr, 'baseline')}")
    print(f"  - Signal Processing: {hasattr(epyr, 'signalprocessing')}")
    print(f"  - Lineshapes: {hasattr(epyr, 'lineshapes')}")
    print(f"  - Plotting: {hasattr(epyr, 'eprplot')}")

## 3. Configuration Management

EPyR Tools uses a hierarchical configuration system that can be managed via CLI or programmatically:

In [ ]:
# Configuration management
from epyr.config import config
import json

print("Current Configuration Settings:")
print("=" * 35)

# Get current configuration
try:
    # Common configuration keys
    config_keys = [
        ('plotting.dpi', 'Plot DPI resolution'),
        ('plotting.figure_size', 'Default figure size'),
        ('performance.cache_enabled', 'Data caching enabled'),
        ('performance.cache_size_mb', 'Cache size (MB)'),
        ('logging.level', 'Logging level'),
        ('data.auto_baseline', 'Auto baseline correction'),
        ('export.format', 'Default export format')
    ]
    
    for key, description in config_keys:
        try:
            value = config.get(key, default="Not set")
            print(f"{description:<25}: {value}")
        except Exception as e:
            print(f"{description:<25}: Error - {e}")
            
except Exception as e:
    print(f"Configuration access error: {e}")

# Demonstrate configuration changes
print("\n" + "="*35)
print("Demonstration: Changing Configuration")
print("="*35)

# Set some configuration values for demonstration
try:
    config.set('plotting.dpi', 300)
    config.set('performance.cache_enabled', True)
    
    print("Updated settings:")
    print(f"Plot DPI: {config.get('plotting.dpi')}")
    print(f"Cache enabled: {config.get('performance.cache_enabled')}")
    
except Exception as e:
    print(f"Could not modify configuration: {e}")

## 4. File Format Conversion and FAIR Data Export

EPyR Tools provides powerful data conversion capabilities for FAIR (Findable, Accessible, Interoperable, Reusable) data standards:

In [ ]:
import epyr
import os
import numpy as np
import matplotlib.pyplot as plt

# First, let's load some EPR data for conversion
data_dir = "../data"

# Find available data files
if os.path.exists(data_dir):
    files = [f for f in os.listdir(data_dir) if f.endswith(('.DTA', '.dta', '.spc'))]
    if files:
        sample_file = os.path.join(data_dir, files[0])
        print(f"Using sample file: {files[0]}")
        
        # Load the data
        try:
            x, y, params, filepath = epyr.eprload(sample_file)
            print(f"Loaded data: {len(y)} points")
            print(f"Field range: {x[0]:.1f} to {x[-1]:.1f} G")
            
            # Show available export formats
            print("\nAvailable Export Formats:")
            print("=" * 30)
            formats = ['CSV', 'JSON', 'HDF5', 'NPZ']
            for fmt in formats:
                print(f"  - {fmt}: Standards-compliant {fmt} format")
            
        except Exception as e:
            print(f"Error loading data: {e}")
            # Create synthetic data for demonstration
            x = np.linspace(3400, 3500, 1000)
            y = np.exp(-((x-3450)/10)**2) + 0.1 * np.random.normal(size=len(x))
            params = {
                'XMIN': x[0],
                'XMAX': x[-1], 
                'XPTS': len(x),
                'YMIN': np.min(y),
                'YMAX': np.max(y),
                'Temperature': '5 K',
                'Microwave_Frequency': '9.5 GHz'
            }
            filepath = "synthetic_demo_data"
            print("Using synthetic data for demonstration")
    else:
        print("No EPR data files found, using synthetic data")
        x = np.linspace(3400, 3500, 1000)
        y = np.exp(-((x-3450)/10)**2) + 0.1 * np.random.normal(size=len(x))
        params = {
            'XMIN': x[0],
            'XMAX': x[-1],
            'XPTS': len(x),
            'Temperature': '5 K',
            'Microwave_Frequency': '9.5 GHz'
        }
        filepath = "synthetic_demo_data"
else:
    print("Data directory not found, using synthetic data")
    x = np.linspace(3400, 3500, 1000)
    y = np.exp(-((x-3450)/10)**2) + 0.1 * np.random.normal(size=len(x))
    params = {
        'XMIN': x[0],
        'XMAX': x[-1],
        'XPTS': len(x),
        'Temperature': '5 K',
        'Microwave_Frequency': '9.5 GHz'
    }
    filepath = "synthetic_demo_data"

In [ ]:
# Demonstrate programmatic data export
import json
import csv
from datetime import datetime

print("FAIR Data Export Demonstration:")
print("=" * 35)

# Create FAIR-compliant metadata
fair_metadata = {
    'data_info': {
        'title': 'EPR Spectroscopy Data',
        'description': 'Electron Paramagnetic Resonance spectrum',
        'creation_date': datetime.now().isoformat(),
        'creator': 'EPyR Tools Tutorial',
        'data_points': len(y),
        'x_axis_label': 'Magnetic Field (G)',
        'y_axis_label': 'Signal Intensity (a.u.)'
    },
    'measurement_parameters': params,
    'processing_info': {
        'software': f'EPyR Tools v{epyr.__version__}',
        'processing_date': datetime.now().isoformat(),
        'baseline_corrected': False,
        'normalized': False
    },
    'data_quality': {
        'signal_to_noise_ratio': np.max(np.abs(y)) / np.std(y[-100:]),
        'data_integrity_check': 'passed'
    }
}

print("Generated FAIR Metadata:")
print(json.dumps(fair_metadata['data_info'], indent=2))
print(f"SNR: {fair_metadata['data_quality']['signal_to_noise_ratio']:.1f}")

# Export to CSV (FAIR format)
csv_filename = '/tmp/epr_data_fair.csv'
try:
    with open(csv_filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        
        # Write metadata header
        writer.writerow(['# EPyR Tools FAIR Data Export'])
        writer.writerow([f'# Creation Date: {fair_metadata["data_info"]["creation_date"]}'])
        writer.writerow([f'# Software: {fair_metadata["processing_info"]["software"]}'])
        writer.writerow(['# Data Format: Field (G), Intensity (a.u.)'])
        writer.writerow([])
        
        # Write column headers
        writer.writerow(['Magnetic_Field_G', 'Signal_Intensity_au'])
        
        # Write data
        for xi, yi in zip(x, y):
            writer.writerow([f'{xi:.6f}', f'{yi:.6e}'])
    
    print(f"\n✅ FAIR CSV export completed: {csv_filename}")
    
except Exception as e:
    print(f"❌ CSV export error: {e}")

# Export metadata to JSON
json_filename = '/tmp/epr_metadata_fair.json'
try:
    with open(json_filename, 'w') as jsonfile:
        json.dump(fair_metadata, jsonfile, indent=2)
    
    print(f"✅ FAIR JSON metadata completed: {json_filename}")
    
except Exception as e:
    print(f"❌ JSON export error: {e}")

## 5. Batch Processing Workflows

EPyR Tools supports efficient batch processing for analyzing multiple files:

In [ ]:
# Batch processing demonstration
import glob
import pandas as pd
from pathlib import Path

print("Batch Processing Demonstration:")
print("=" * 35)

# Find all EPR data files
data_patterns = ["../data/*.DTA", "../data/*.dta", "../data/*.spc"]
all_files = []

for pattern in data_patterns:
    all_files.extend(glob.glob(pattern))

if all_files:
    print(f"Found {len(all_files)} EPR data files for batch processing:")
    for file in all_files[:5]:  # Show first 5
        print(f"  - {Path(file).name}")
    if len(all_files) > 5:
        print(f"  ... and {len(all_files)-5} more files")
else:
    print("No EPR data files found. Creating synthetic batch for demonstration.")
    # Create synthetic batch data
    all_files = ["synthetic_file_1.DTA", "synthetic_file_2.DTA", "synthetic_file_3.DTA"]

# Batch processing function
def process_epr_file_batch(filepath, process_baseline=True, export_formats=['CSV']):
    """Process a single EPR file with configurable options"""
    results = {
        'filename': Path(filepath).name,
        'status': 'pending',
        'data_points': 0,
        'field_range': (0, 0),
        'signal_max': 0,
        'signal_min': 0,
        'snr_estimate': 0,
        'processing_time': 0,
        'exports': []
    }
    
    try:
        start_time = datetime.now()
        
        if filepath.startswith('synthetic'):
            # Generate synthetic data for demonstration
            x = np.linspace(3400, 3500, 1000) + np.random.normal(0, 50)
            y = np.exp(-((x-3450)/15)**2) + 0.1 * np.random.normal(size=len(x))
            params = {'Temperature': f'{np.random.randint(4,20)} K'}
        else:
            # Load real data
            x, y, params, _ = epyr.eprload(filepath)
        
        # Basic analysis
        results.update({
            'status': 'success',
            'data_points': len(y),
            'field_range': (float(x[0]), float(x[-1])),
            'signal_max': float(np.max(y)),
            'signal_min': float(np.min(y)),
            'snr_estimate': float(np.max(np.abs(y)) / np.std(y[-100:]))
        })
        
        # Optional baseline correction
        if process_baseline:
            try:
                y_corrected = epyr.baseline.correct_1d(x, y, order=2)
                results['baseline_corrected'] = True
                y = y_corrected  # Use corrected data for export
            except:
                results['baseline_corrected'] = False
        
        # Export in requested formats
        for fmt in export_formats:
            export_file = f"/tmp/{Path(filepath).stem}_processed.{fmt.lower()}"
            try:
                if fmt == 'CSV':
                    np.savetxt(export_file, np.column_stack([x, y]), 
                              delimiter=',', header='Field_G,Intensity_au')
                elif fmt == 'NPZ':
                    np.savez(export_file, x=x, y=y, params=params)
                results['exports'].append(export_file)
            except Exception as e:
                results['export_error'] = str(e)
        
        # Calculate processing time
        results['processing_time'] = (datetime.now() - start_time).total_seconds()
        
    except Exception as e:
        results.update({
            'status': 'error',
            'error_message': str(e)
        })
    
    return results

# Process batch of files
print("\nProcessing files...")
batch_results = []

for filepath in all_files[:3]:  # Process first 3 files
    print(f"Processing: {Path(filepath).name}")
    result = process_epr_file_batch(filepath, process_baseline=True, export_formats=['CSV'])
    batch_results.append(result)
    print(f"  Status: {result['status']} ({result['processing_time']:.2f}s)")

# Create batch processing summary
print("\n" + "="*40)
print("Batch Processing Summary")
print("="*40)

df_results = pd.DataFrame(batch_results)
print(f"Total files processed: {len(batch_results)}")
print(f"Successful: {sum(df_results['status'] == 'success')}")
print(f"Failed: {sum(df_results['status'] == 'error')}")
print(f"Average processing time: {df_results['processing_time'].mean():.2f}s")
print(f"Total data points processed: {df_results['data_points'].sum()}")

if len(df_results) > 0:
    print("\nFile Summary:")
    for _, row in df_results.iterrows():
        print(f"  {row['filename']:<30} {row['data_points']:>6} pts, SNR: {row['snr_estimate']:.1f}")

## 6. Performance Optimization and Caching

EPyR Tools includes sophisticated performance optimization features:

In [ ]:
# Performance optimization demonstration
import time
import psutil
import gc

print("Performance Optimization Demonstration:")
print("=" * 45)

# Memory usage monitoring
def get_memory_usage():
    """Get current memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

initial_memory = get_memory_usage()
print(f"Initial memory usage: {initial_memory:.1f} MB")

# Demonstrate data caching
print("\n1. Data Loading Performance:")
print("-" * 30)

# Create large synthetic dataset for performance testing
large_x = np.linspace(0, 1000, 100000)  # 100k points
large_y = np.sin(large_x) + 0.1 * np.random.normal(size=len(large_x))

print(f"Created large dataset: {len(large_y):,} points")
print(f"Memory after data creation: {get_memory_usage():.1f} MB (+{get_memory_usage()-initial_memory:.1f} MB)")

# Simulate data processing with timing
processing_times = []
memory_usage = []

for i in range(3):
    start_time = time.time()
    start_memory = get_memory_usage()
    
    # Simulate expensive processing
    processed_data = np.fft.fft(large_y)
    smoothed_data = np.convolve(large_y, np.ones(10)/10, mode='same')
    
    end_time = time.time()
    end_memory = get_memory_usage()
    
    processing_times.append(end_time - start_time)
    memory_usage.append(end_memory - start_memory)
    
    print(f"Processing run {i+1}: {processing_times[i]:.3f}s, +{memory_usage[i]:.1f} MB")
    
    # Clean up intermediate results
    del processed_data, smoothed_data
    gc.collect()

print(f"Average processing time: {np.mean(processing_times):.3f}s ± {np.std(processing_times):.3f}s")
print(f"Average memory overhead: {np.mean(memory_usage):.1f} MB")

# Demonstrate configuration impact on performance
print("\n2. Configuration Impact on Performance:")
print("-" * 42)

# Test different DPI settings for plotting
dpi_settings = [72, 150, 300]
plot_times = []

for dpi in dpi_settings:
    config.set('plotting.dpi', dpi)
    
    start_time = time.time()
    
    # Create plot with current DPI setting
    fig, ax = plt.subplots(figsize=(8, 6), dpi=dpi)
    ax.plot(large_x[::100], large_y[::100])  # Subsample for plotting
    ax.set_title(f'Plot at {dpi} DPI')
    plt.close(fig)  # Close to save memory
    
    plot_time = time.time() - start_time
    plot_times.append(plot_time)
    
    print(f"Plot at {dpi:>3} DPI: {plot_time:.3f}s")

print(f"\nPerformance recommendation: Use DPI ≤ 150 for interactive work")
print(f"High DPI (300) is {plot_times[2]/plot_times[0]:.1f}x slower than low DPI (72)")

# Clean up large dataset
del large_x, large_y
gc.collect()

final_memory = get_memory_usage()
print(f"\nFinal memory usage: {final_memory:.1f} MB (change: {final_memory-initial_memory:+.1f} MB)")

## 7. Plugin Architecture and Extensibility

EPyR Tools is designed with a plugin architecture for easy extensibility:

In [ ]:
# Plugin architecture demonstration
print("Plugin Architecture Demonstration:")
print("=" * 38)

# Show current plugin system status
try:
    from epyr.plugins import plugin_manager
    
    print("Plugin Manager Status:")
    print(f"  Available plugin slots: File loaders, processors, exporters")
    print(f"  Auto-discovery enabled: Yes")
    print(f"  Entry point namespace: 'epyr_plugins'")
    
except ImportError:
    print("Plugin manager not available in this installation")

# Demonstrate a simple custom processing function
print("\nCustom Processing Function Example:")
print("-" * 38)

def custom_noise_analysis(x, y, params=None):
    """
    Custom EPR data processing: Advanced noise analysis
    
    This function demonstrates how to create custom processing
    that could be packaged as an EPyR Tools plugin.
    """
    results = {
        'function': 'custom_noise_analysis',
        'version': '1.0.0',
        'description': 'Advanced noise characterization for EPR data'
    }
    
    # Calculate various noise metrics
    signal = np.abs(y)
    
    # Estimate signal regions (above 10% of max)
    signal_threshold = 0.1 * np.max(signal)
    signal_region = signal > signal_threshold
    noise_region = ~signal_region
    
    # Noise statistics
    if np.any(noise_region):
        noise_std = np.std(y[noise_region])
        noise_mean = np.mean(y[noise_region])
    else:
        # Fallback: use last 10% of data as noise estimate
        noise_end = int(0.9 * len(y))
        noise_std = np.std(y[noise_end:])
        noise_mean = np.mean(y[noise_end:])
    
    # Signal quality metrics
    peak_signal = np.max(signal)
    snr = peak_signal / noise_std if noise_std > 0 else np.inf
    
    # Frequency domain noise analysis
    freq_spectrum = np.abs(np.fft.fft(y))
    freq_noise_floor = np.median(freq_spectrum[len(freq_spectrum)//4:])
    
    # Dynamic range
    dynamic_range = 20 * np.log10(peak_signal / noise_std) if noise_std > 0 else np.inf
    
    results.update({
        'noise_statistics': {
            'std_deviation': float(noise_std),
            'mean_offset': float(noise_mean),
            'freq_noise_floor': float(freq_noise_floor)
        },
        'signal_quality': {
            'snr_linear': float(snr),
            'snr_db': float(20 * np.log10(snr)) if snr > 0 else -np.inf,
            'dynamic_range_db': float(dynamic_range),
            'peak_signal': float(peak_signal)
        },
        'data_quality_grade': 'A' if snr > 50 else 'B' if snr > 20 else 'C' if snr > 10 else 'D'
    })
    
    return results

# Test the custom function
test_x = np.linspace(3400, 3500, 1000)
test_y = np.exp(-((test_x-3450)/10)**2) + 0.05 * np.random.normal(size=len(test_x))

custom_results = custom_noise_analysis(test_x, test_y)

print(f"Custom Analysis Results:")
print(f"  Function: {custom_results['function']} v{custom_results['version']}")
print(f"  SNR: {custom_results['signal_quality']['snr_linear']:.1f} ({custom_results['signal_quality']['snr_db']:.1f} dB)")
print(f"  Dynamic Range: {custom_results['signal_quality']['dynamic_range_db']:.1f} dB")
print(f"  Data Quality Grade: {custom_results['data_quality_grade']}")
print(f"  Noise Statistics: σ = {custom_results['noise_statistics']['std_deviation']:.2e}")

# Show how this could be integrated into EPyR Tools workflow
print("\nIntegration Example:")
print("-------------------")
print("# To create an EPyR Tools plugin:")
print("# 1. Create a Python package with the processing function")
print("# 2. Add entry point in setup.py:")
print("#    entry_points={")
print("#        'epyr_plugins': [")
print("#            'noise_analyzer = my_plugin:custom_noise_analysis'")
print("#        ]")
print("#    }")
print("# 3. Install the plugin package")
print("# 4. EPyR Tools will auto-discover and load the plugin")

## 8. Integration Workflows

EPyR Tools is designed to integrate seamlessly with other scientific software:

In [ ]:
# Integration workflow demonstration
import json
import subprocess
import tempfile
from pathlib import Path

print("Integration Workflows Demonstration:")
print("=" * 40)

# 1. Integration with common Python scientific stack
print("1. Scientific Python Stack Integration:")
print("-" * 40)

# Generate example data
x = np.linspace(3400, 3500, 1000)
y = np.exp(-((x-3450)/15)**2) + 0.1 * np.random.normal(size=len(x))

# Integration with pandas
try:
    import pandas as pd
    
    df = pd.DataFrame({
        'field_g': x,
        'intensity': y,
        'normalized_intensity': y / np.max(np.abs(y))
    })
    
    print(f"✅ Pandas integration: {len(df)} data points in DataFrame")
    print(f"   Data summary: {df['intensity'].describe().to_dict()}")
    
except ImportError:
    print("❌ Pandas not available")

# Integration with scipy
try:
    from scipy import signal, optimize
    
    # Apply scipy filters
    filtered_y = signal.savgol_filter(y, window_length=21, polyorder=3)
    
    # Find peaks using scipy
    peaks, properties = signal.find_peaks(np.abs(filtered_y), height=0.1*np.max(np.abs(filtered_y)))
    
    print(f"✅ SciPy integration: {len(peaks)} peaks detected")
    if len(peaks) > 0:
        peak_fields = x[peaks]
        print(f"   Peak positions: {[f'{p:.1f} G' for p in peak_fields]}")
    
except ImportError:
    print("❌ SciPy not available")

# 2. File format conversions for other software
print("\n2. External Software Format Export:")
print("-" * 37)

# Export for Origin/OriginPro
origin_file = '/tmp/epr_data_for_origin.dat'
try:
    with open(origin_file, 'w') as f:
        f.write("# EPyR Tools export for Origin\n")
        f.write("# Field(G)\tIntensity(a.u.)\n")
        for xi, yi in zip(x, y):
            f.write(f"{xi:.6f}\t{yi:.6e}\n")
    print(f"✅ Origin format: {origin_file}")
except Exception as e:
    print(f"❌ Origin export error: {e}")

# Export for MATLAB
matlab_file = '/tmp/epr_data_for_matlab.m'
try:
    with open(matlab_file, 'w') as f:
        f.write("% EPyR Tools export for MATLAB\n")
        f.write("% Generated by EPyR Tools\n\n")
        f.write("field_g = [")
        f.write(", ".join([f"{xi:.6f}" for xi in x]))
        f.write("];\n\n")
        f.write("intensity = [")
        f.write(", ".join([f"{yi:.6e}" for yi in y]))
        f.write("];\n\n")
        f.write("% Plot the data\n")
        f.write("figure;\n")
        f.write("plot(field_g, intensity);\n")
        f.write("xlabel('Magnetic Field (G)');\n")
        f.write("ylabel('Intensity (a.u.)');\n")
        f.write("title('EPR Spectrum');\n")
    print(f"✅ MATLAB format: {matlab_file}")
except Exception as e:
    print(f"❌ MATLAB export error: {e}")

# 3. Command-line workflow integration
print("\n3. Command-Line Workflow Integration:")
print("-" * 38)

# Create a shell script for automated processing
shell_script = '/tmp/epr_processing_pipeline.sh'
try:
    script_content = '''#!/bin/bash
# EPyR Tools Automated Processing Pipeline
# Usage: ./epr_processing_pipeline.sh <data_directory>

DATA_DIR="${1:-../data}"
OUTPUT_DIR="/tmp/epr_batch_output"

echo "EPyR Tools Batch Processing Pipeline"
echo "===================================="
echo "Input directory: $DATA_DIR"
echo "Output directory: $OUTPUT_DIR"

# Create output directory
mkdir -p "$OUTPUT_DIR"

# Process each EPR file
for file in "$DATA_DIR"/*.{DTA,dta,spc}; do
    if [ -f "$file" ]; then
        echo "Processing: $(basename "$file")"
        
        # Load and convert to CSV (using hypothetical CLI command)
        # epyr-convert "$file" --format CSV --output "$OUTPUT_DIR"
        
        # Apply baseline correction
        # epyr-baseline "$file" --order 2 --output "$OUTPUT_DIR"
        
        # Generate plot
        # epyr-plot "$file" --save "$OUTPUT_DIR/$(basename "$file" .DTA).png"
        
        echo "  -> Processed successfully"
    fi
done

echo "\nBatch processing completed!"
echo "Results saved to: $OUTPUT_DIR"
'''
    
    with open(shell_script, 'w') as f:
        f.write(script_content)
    
    # Make executable
    os.chmod(shell_script, 0o755)
    
    print(f"✅ Shell script created: {shell_script}")
    print(f"   Usage: bash {shell_script} <data_directory>")
    
except Exception as e:
    print(f"❌ Shell script error: {e}")

# 4. Python API integration example
print("\n4. Python API Integration Example:")
print("-" * 35)

# Create a complete processing workflow function
def complete_epr_workflow(data_file, output_dir='/tmp/epr_workflow_output'):
    """
    Complete EPR data processing workflow suitable for integration
    into larger scientific analysis pipelines.
    """
    workflow_results = {
        'input_file': data_file,
        'output_directory': output_dir,
        'processing_steps': [],
        'outputs': [],
        'status': 'success'
    }
    
    try:
        # Create output directory
        os.makedirs(output_dir, exist_ok=True)
        
        # Step 1: Load data
        if data_file.startswith('synthetic'):
            x, y = test_x, test_y
            params = {'Temperature': '5 K'}
        else:
            x, y, params, _ = epyr.eprload(data_file)
        
        workflow_results['processing_steps'].append('data_loaded')
        
        # Step 2: Baseline correction
        try:
            y_corrected = epyr.baseline.correct_1d(x, y, order=2)
            workflow_results['processing_steps'].append('baseline_corrected')
            y = y_corrected
        except:
            workflow_results['processing_steps'].append('baseline_correction_failed')
        
        # Step 3: Export processed data
        output_file = os.path.join(output_dir, 'processed_data.csv')
        np.savetxt(output_file, np.column_stack([x, y]), 
                  delimiter=',', header='Field_G,Intensity_au')
        workflow_results['outputs'].append(output_file)
        
        # Step 4: Generate summary plot
        fig, ax = plt.subplots(figsize=(10, 6))
        ax.plot(x, y, 'b-', linewidth=1.5, label='Processed Data')
        ax.set_xlabel('Magnetic Field (G)')
        ax.set_ylabel('Intensity (a.u.)')
        ax.set_title('EPR Spectrum - Processed')
        ax.grid(True, alpha=0.3)
        ax.legend()
        
        plot_file = os.path.join(output_dir, 'spectrum_plot.png')
        fig.savefig(plot_file, dpi=150, bbox_inches='tight')
        plt.close(fig)
        workflow_results['outputs'].append(plot_file)
        
        # Step 5: Generate metadata
        metadata = {
            'processing_info': {
                'software': f'EPyR Tools v{epyr.__version__}',
                'timestamp': datetime.now().isoformat(),
                'workflow_version': '1.0'
            },
            'data_info': {
                'data_points': len(y),
                'field_range_g': [float(x[0]), float(x[-1])],
                'signal_range': [float(np.min(y)), float(np.max(y))]
            },
            'parameters': params
        }
        
        metadata_file = os.path.join(output_dir, 'metadata.json')
        with open(metadata_file, 'w') as f:
            json.dump(metadata, f, indent=2)
        workflow_results['outputs'].append(metadata_file)
        
        workflow_results['processing_steps'].append('workflow_completed')
        
    except Exception as e:
        workflow_results['status'] = 'error'
        workflow_results['error_message'] = str(e)
    
    return workflow_results

# Test the complete workflow
workflow_result = complete_epr_workflow('synthetic_demo_file.DTA')

print(f"Workflow Status: {workflow_result['status']}")
print(f"Processing Steps: {' → '.join(workflow_result['processing_steps'])}")
print(f"Output Files: {len(workflow_result['outputs'])}")
for output in workflow_result['outputs']:
    print(f"  - {Path(output).name}")

print("\n✅ Integration workflows demonstrated successfully!")
print("   These examples show how EPyR Tools can be integrated")
print("   into larger scientific computing workflows and pipelines.")

## 9. Advanced CLI Usage Examples

Here are practical examples of using EPyR Tools CLI commands for real-world scenarios:

In [ ]:
# Advanced CLI usage examples
print("Advanced CLI Usage Examples:")
print("=" * 32)

# Generate example CLI command templates
cli_examples = {
    'Batch Data Conversion': [
        '# Convert all EPR files in a directory to CSV format',
        'epyr-batch convert --input-dir ./data --output-dir ./converted --format CSV',
        '',
        '# Convert with baseline correction applied',
        'epyr-batch convert --input-dir ./data --baseline-order 2 --format JSON'
    ],
    
    'Automated Baseline Correction': [
        '# Apply polynomial baseline correction to all files',
        'epyr-batch baseline --input-pattern "*.DTA" --order 3 --save-corrected',
        '',
        '# Interactive baseline correction with preview',
        'epyr-baseline data.DTA --interactive --preview'
    ],
    
    'Plotting and Visualization': [
        '# Generate publication-quality plots',
        'epyr-plot data.DTA --dpi 300 --format PNG --style publication',
        '',
        '# Create comparison plot of multiple files',
        'epyr-plot file1.DTA file2.DTA file3.DTA --overlay --normalize'
    ],
    
    'Data Validation and Quality Check': [
        '# Validate data integrity and FAIR compliance',
        'epyr-validate --input-dir ./data --check-integrity --fair-compliance',
        '',
        '# Generate data quality report',
        'epyr-validate data.DTA --quality-report --output report.json'
    ],
    
    'System Configuration': [
        '# Show current configuration',
        'epyr-config show',
        '',
        '# Set performance options',
        'epyr-config set performance.cache_enabled true',
        'epyr-config set plotting.dpi 150',
        '',
        '# Reset to defaults',
        'epyr-config reset'
    ]
}

# Display CLI examples
for category, commands in cli_examples.items():
    print(f"\n{category}:")
    print("-" * (len(category) + 1))
    for cmd in commands:
        if cmd.startswith('#'):
            print(f"  {cmd}")
        elif cmd.strip() == '':
            print()
        else:
            print(f"  $ {cmd}")

# Create a comprehensive CLI cheat sheet
print("\n" + "="*50)
print("EPyR Tools CLI Quick Reference")
print("="*50)

cheat_sheet = '''## Essential Commands

**System Info & Diagnostics:**
epyr-info                    # System information
epyr-info --all             # Detailed diagnostics
epyr-config show            # Current configuration

**Data Loading & Inspection:**
epyr-load data.DTA          # Load and display data info
epyr-load data.DTA --plot   # Load and plot data

**Baseline Correction:**
epyr-baseline data.DTA --order 2        # Polynomial correction
epyr-baseline data.DTA --interactive    # Interactive mode

**Data Conversion (FAIR):**
epyr-convert data.DTA --format CSV      # Convert to CSV
epyr-convert data.DTA --format JSON     # Convert to JSON
epyr-convert data.DTA --format HDF5     # Convert to HDF5

**Plotting & Visualization:**
epyr-plot data.DTA                      # Quick plot
epyr-plot data.DTA --save plot.png     # Save plot
epyr-plot data.DTA --dpi 300           # High resolution

**Batch Processing:**
epyr-batch process --input-dir ./data  # Batch processing
epyr-batch convert --format CSV        # Batch conversion

**Data Validation:**
epyr-validate data.DTA                 # Basic validation
epyr-validate --fair-compliance        # FAIR compliance check

## Advanced Usage Patterns

**Pipeline Processing:**
# Load → Baseline correct → Convert → Plot
epyr-load data.DTA | epyr-baseline --order 2 | epyr-convert --format CSV | epyr-plot

**Directory Processing:**
# Process all .DTA files in current directory
find . -name "*.DTA" -exec epyr-load {} \;

**Configuration Management:**
epyr-config set plotting.style publication  # Set publication style
epyr-config set performance.parallel true   # Enable parallel processing
'''

print(cheat_sheet)

## 10. Best Practices and Performance Tips

Here are key recommendations for optimal use of EPyR Tools:

In [ ]:
# Best practices demonstration
print("EPyR Tools Best Practices and Performance Tips:")
print("=" * 48)

# Performance recommendations
performance_tips = {
    'Memory Management': [
        'Enable data caching for repeated analysis: config.set("performance.cache_enabled", True)',
        'Use streaming mode for large files (>100 MB): epyr.eprload(file, stream=True)',
        'Clear cache periodically in long-running scripts: epyr.clear_cache()',
        'Monitor memory usage with: epyr-info --memory'
    ],
    
    'Data Loading Optimization': [
        'Use format hints for faster loading: epyr.eprload(file, format="BES3T")',
        'Batch load similar files together for better cache utilization',
        'Pre-validate file paths to avoid loading errors: epyr.validate_file(path)',
        'Use memory mapping for very large datasets: load_options={"mmap": True}'
    ],
    
    'Plotting Performance': [
        'Use appropriate DPI: 72 for screen, 150 for preview, 300 for publication',
        'Subsample large datasets for plotting: x[::10], y[::10]',
        'Close figures after saving: plt.close(fig)',
        'Use vector formats (SVG, PDF) for final publications'
    ],
    
    'Baseline Correction': [
        'Start with low polynomial order (1-2) and increase if needed',
        'Use interactive mode for critical data: epyr.baseline.interactive()',
        'Save baseline parameters for reproducibility',
        'Validate correction quality with SNR metrics'
    ],
    
    'Signal Processing': [
        'Always remove DC offset before FFT: remove_dc=True',
        'Choose appropriate window function: Hann for general use',
        'Use zero padding for better frequency resolution: zero_padding=2',
        'Consider sampling rate when interpreting frequencies'
    ],
    
    'Data Export and FAIR Compliance': [
        'Always include comprehensive metadata in exports',
        'Use standardized units and parameter names',
        'Include processing history and software versions',
        'Validate exported data integrity: epyr-validate output.csv'
    ],
    
    'Error Handling': [
        'Always wrap file operations in try-except blocks',
        'Check file formats before processing: epyr.detect_format(file)',
        'Validate parameter ranges before analysis',
        'Use logging for debugging: import logging; logging.basicConfig(level=logging.DEBUG)'
    ],
    
    'Reproducibility': [
        'Save configuration settings: epyr-config export settings.json',
        'Version control analysis scripts and parameters',
        'Document software versions: epyr-info > system_info.txt',
        'Use deterministic random seeds for synthetic data'
    ]
}

# Display best practices
for category, tips in performance_tips.items():
    print(f"\n{category}:")
    print("-" * (len(category) + 1))
    for tip in tips:
        print(f"  • {tip}")

# Demonstrate configuration for optimal performance
print("\n" + "="*40)
print("Recommended Configuration for Different Use Cases")
print("="*40)

use_cases = {
    'Interactive Analysis': {
        'plotting.dpi': 72,
        'performance.cache_enabled': True,
        'performance.cache_size_mb': 256,
        'logging.level': 'INFO'
    },
    
    'Batch Processing': {
        'plotting.dpi': 150,
        'performance.cache_enabled': True,
        'performance.cache_size_mb': 512,
        'performance.parallel': True,
        'logging.level': 'WARNING'
    },
    
    'Publication Quality': {
        'plotting.dpi': 300,
        'plotting.style': 'publication',
        'export.include_metadata': True,
        'export.fair_compliance': True,
        'logging.level': 'INFO'
    }
}

for use_case, settings in use_cases.items():
    print(f"\n{use_case}:")
    for key, value in settings.items():
        print(f"  {key}: {value}")

print("\n" + "="*40)
print("Performance Benchmarking")
print("="*40)

# Simple performance benchmarking
def benchmark_operation(operation_name, operation_func, *args, **kwargs):
    """Benchmark an EPyR Tools operation"""
    start_time = time.time()
    start_memory = get_memory_usage()
    
    try:
        result = operation_func(*args, **kwargs)
        success = True
    except Exception as e:
        result = str(e)
        success = False
    
    end_time = time.time()
    end_memory = get_memory_usage()
    
    return {
        'operation': operation_name,
        'success': success,
        'duration_s': end_time - start_time,
        'memory_change_mb': end_memory - start_memory,
        'result_summary': str(result)[:50] if success else result
    }

# Benchmark common operations
test_x = np.linspace(3400, 3500, 10000)  # 10k points
test_y = np.exp(-((test_x-3450)/10)**2) + 0.05 * np.random.normal(size=len(test_x))

benchmarks = []
benchmarks.append(benchmark_operation(
    "Baseline Correction", 
    lambda: epyr.baseline.correct_1d(test_x, test_y, order=2)
))

benchmarks.append(benchmark_operation(
    "Signal Processing FFT", 
    lambda: epyr.signalprocessing.analyze_frequencies(test_x, test_y, plot=False)
))

benchmarks.append(benchmark_operation(
    "Lineshape Fitting", 
    lambda: epyr.lineshapes.gaussian(test_x, center=3450, width=10)
))

print("\nBenchmark Results:")
print(f"{'Operation':<20} {'Time (s)':<10} {'Memory (MB)':<12} {'Status':<8}")
print("-" * 55)

for bench in benchmarks:
    status = "✅ OK" if bench['success'] else "❌ Error"
    print(f"{bench['operation']:<20} {bench['duration_s']:<10.3f} {bench['memory_change_mb']:<12.1f} {status:<8}")

print("\n✅ EPyR Tools Advanced Features Tutorial Completed!")
print("\nYou now have comprehensive knowledge of:")
print("  • Command-line interface and automation")
print("  • Batch processing and FAIR data export")
print("  • Performance optimization techniques")
print("  • Integration with other scientific tools")
print("  • Best practices for reproducible research")

## Summary and Next Steps

Congratulations! You have completed the EPyR Tools tutorial series. You now have comprehensive knowledge of:

### **Tutorial Series Review:**

1. **[Tutorial 01](01_Basic_EPR_Data_Loading.ipynb)**: EPR data loading from Bruker formats (BES3T, ESP)
2. **[Tutorial 02](02_Baseline_Correction_Guide.ipynb)**: Polynomial baseline correction techniques
3. **[Tutorial 03](03_Signal_Processing_Analysis.ipynb)**: FFT-based signal processing and frequency analysis
4. **[Tutorial 04](04_Lineshape_Analysis_Fitting.ipynb)**: EPR lineshape fitting and spectral analysis
5. **[Tutorial 05](05_Advanced_Features_CLI.ipynb)**: CLI tools, automation, and integration workflows

### **Key Capabilities Mastered:**

- **Data Management**: Loading, validating, and exporting EPR data
- **Signal Processing**: FFT analysis, noise characterization, frequency detection
- **Spectral Analysis**: Baseline correction, lineshape fitting, parameter extraction
- **Automation**: Batch processing, CLI workflows, configuration management
- **Integration**: FAIR data compliance, external software compatibility
- **Performance**: Memory optimization, caching, benchmarking

### **Next Steps:**

1. **Practice**: Apply these techniques to your own EPR data
2. **Explore**: Experiment with different parameters and methods
3. **Integrate**: Incorporate EPyR Tools into your research workflow
4. **Contribute**: Share feedback and contribute to the project
5. **Stay Updated**: Follow the project for new features and improvements

### **Resources:**

- **Documentation**: https://epyr-tools.readthedocs.io/
- **GitHub**: https://github.com/BertainaS/epyrtools
- **Issues & Support**: https://github.com/BertainaS/epyrtools/issues
- **PyPI**: https://pypi.org/project/epyr-tools/

### **Citation:**

If you use EPyR Tools in your research, please cite:
```
EPyR Tools: A Python Package for EPR Spectroscopy Data Analysis
Version 0.2.0+
https://github.com/BertainaS/epyrtools
```

---

**Thank you for using EPyR Tools!** 

We hope this tutorial series has provided you with the knowledge and confidence to perform sophisticated EPR data analysis using Python. Happy analyzing! 🔬